In [1]:
from sklearn.externals import joblib
import gc
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch
import numpy as np
import swifter
import torch.optim as optim
import pandas as pd
import time

/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.errors', please update to use 'numba.core.errors' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.errors import TypingError
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has m

# CONFIGURATION

In [2]:
config={
    'lstm_dropout':0.0,
    'char_embedding_size':600,
    'hidden_dims':384,
    'rnn_layers':1,
    'conv_in_dim':600,
    'conv_out_dim':128,
    'conv_kernel_size':3,
    'img_vec_dim':2053,
    'transformer_dim':768,
    'img_transformer_dim':2048+5,
    'encoder_layers':6,
    'img_encoder_layers':3,
    'heads':8,
    'ffn_dim':2048,
    'transformer_dropout':0.0
}

In [3]:
MAX_BOX = 10
MAX_QUERY_CHAR = 12

# MODEL

In [4]:
class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        self.keep_dropout = config['lstm_dropout']
        self.char_embedding_size = config['char_embedding_size']
        self.hidden_dims = config['hidden_dims']
        self.rnn_layers = config['rnn_layers']

        self.conv_in_dim = config['conv_in_dim']
        self.conv_out_dim = config['conv_out_dim']
        self.conv_kernel_size = config['conv_kernel_size']

        self.dense_input_dim = config['img_vec_dim']
        self.dense_output_dim = config['model_dim']
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=config['transformer_dim'], 
                                                   nhead=config['heads'],
                                                   dim_feedforward=config['ffn_dim'],
                                                   dropout =0.0
                                                  )
        
        img_encoder_layer = nn.TransformerEncoderLayer(d_model=config['img_transformer_dim'], 
                                                   nhead=config['heads'],
                                                   dim_feedforward=config['ffn_dim'],
                                                   dropout =0.0
                                                  )
        
        self.query_img_transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=config['encoder_layers'])
        self.img_transformer_encoder = nn.TransformerEncoder(img_encoder_layer, num_layers=config['img_encoder_layers'])

        self.lstm_net_query = nn.LSTM(self.char_embedding_size, self.hidden_dims,
                                      num_layers=self.rnn_layers, dropout=self.keep_dropout,
                                      bidirectional=True, batch_first=True)
        self.conv1d = nn.Conv1d(in_channels=self.conv_in_dim, out_channels=self.conv_out_dim,
                                kernel_size=self.conv_kernel_size)
#         self.convs = nn.Sequential(self.conv1d, F.avg_pool1d(kernel_size=3, stride=1),nn.ReLU())
        self.linearLayer = nn.Linear(self.dense_input_dim, self.dense_output_dim)

        self.mlpLayer = nn.Sequential(nn.BatchNorm1d(768),nn.ReLU(), nn.Linear(768, 256), 
                                      nn.BatchNorm1d(256),nn.ReLU(), nn.Linear(256, 64), 
                                      nn.BatchNorm1d(64), nn.ReLU(),
                                      nn.Linear(64, 1))

    def forward(self, query_vec, label_text_vec, query_bert_vec, img_vec, pos_vec, padding_mask,img_padding_mask,lstm_seq_length):
        pack = nn.utils.rnn.pack_padded_sequence(query_vec, lstm_seq_length, batch_first=True,enforce_sorted=False)
        lstm_output, (h, c) = self.lstm_net_query(pack)
        h = h.permute(1, 0, 2)
        h = torch.cat((h[:,0,:],h[:,1,:]),dim=1)
        query_bert_vec = query_bert_vec + h.view(query_bert_vec.shape[0], query_bert_vec.shape[1], query_bert_vec.shape[2])
#         print('query_bert_vec size:', query_bert_vec.shape)
        label_text_vec = label_text_vec.permute(0, 2, 1)
#         print('label_text_vec size:', label_text_vec.shape)
#         conv_output = self.convs(label_text_vec)
        conv_output = self.conv1d(label_text_vec)
        conv_output = F.avg_pool1d(conv_output, kernel_size=3, stride=1)
#         print(conv_output.shape)
        conv_output = conv_output.view(conv_output.shape[0], 1, conv_output.shape[1]*conv_output.shape[2])
#         print('conv_output size:', conv_output.shape)
#         print(img_vec.shape, pos_vec.shape)
        img_vec = torch.cat((img_vec, pos_vec), dim=2)
#         print(img_vec.shape)
        img_outout = self.img_transformer_encoder(img_vec, src_key_padding_mask=img_padding_mask)

        img_output = nn.functional.relu(self.linearLayer(img_vec))
#         print('img_output size:', img_output.shape)
#         print('query_bert_vec size:', query_bert_vec.shape)
#         print('conv_output size:', conv_output.shape)
        output = torch.cat((query_bert_vec, img_output), dim=1)
#         print('output size:', output.shape)
        output = output.permute(1, 0 ,2)
        output = self.transformer_encoder(output, src_key_padding_mask=padding_mask).permute(1, 0 ,2)
        output = output[:,0,:]
        output = self.mlpLayer(output)
#         print('output size:', output.shape)
        return output


In [5]:
# loss function
# loss_f = nn.BCEWithLogitsLoss()
sigmoid=nn.Sigmoid()
def loss_f(predict, label):
    predict=sigmoid(predict)
    return (-label*torch.log(predict)-(1-label)*torch.log((1-predict)*0.6)).mean()

def calc_loss(predict, label):
    loss = loss_f(predict.squeeze(dim=-1), label)
    return loss

In [6]:
# generate neg samples
from sklearn.utils import shuffle

# def generate_negative_sample(train):
#     train.reset_index(drop=True, inplace=True)
#     df1 = shuffle(train, random_state=2020)
#     df1.reset_index(drop=True, inplace=True)
#     df1['old_query_id'] = train['query_id']
#     df1['old_product_id'] = train['product_id']
#     df1['query_albert_embedding'] = train['query_albert_embedding']
#     df1['query_embedding'] = train['query_embedding']
#     df['num_query_char'] = train['num_query_char']

#     df1['label'] = df1.apply(lambda x: False if x['query_id']!=x['old_query_id']
#                                                         and x['product_id']!=x['old_product_id'] else True, axis=1)

#     output = df1[df1['label']==False]
#     output['label']=0
#     output = output[train.columns]
#     print('negative sample number:', output.shape[0])
#     del df1
#     gc.collect()
#     return output

def generate_negative_sample(train, num):
    train.reset_index(drop=True, inplace=True)
    output=pd.DataFrame()
    for i in range(0, num):
        df = shuffle(train, random_state=2020+i)
        df.reset_index(drop=True, inplace=True)
        df['old_query_id'] = train['query_id']
        df['old_product_id'] = train['product_id']
        df['query_albert_embedding'] = train['query_albert_embedding']
        df['query_embedding'] = train['query_embedding']
        df['num_query_char'] = train['num_query_char']

        df['label'] = df.apply(lambda x: False if x['query_id']!=x['old_query_id']
                                                            and x['product_id']!=x['old_product_id'] else True, axis=1)

        df = df[df['label']==False]
        df['label']=0
        df = df[train.columns]
        
        output = pd.concat((output, df), axis=0)
    
    output.reset_index(drop=True, inplace=True)
    print('negative sample number:', output.shape[0])
    del df
    gc.collect()
    return output


# TRAIN DATA

In [7]:
# load 100000 samples 
def load_train_data(train_albert_path, train_data_path):
    train_albert_embedded=joblib.load(train_albert_path)
    train=joblib.load(train_data_path)
    
    train.reset_index(drop=True, inplace=True)
    train_albert_embedded.reset_index(drop=True, inplace=True)
    
    # truncate
    train['label_embedding']=train.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
    train['pos']=train.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
    train['query_embedding']=train.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
    train['num_box']=train.swifter.apply(lambda x: len(x['pos']),axis=1)
    train['num_query_char']=train.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
    train['features']=train.swifter.apply(lambda x: [i for i in x['features']],axis=1)
    train['features']=train.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
    train['query_albert_embedding']=train_albert_embedded[0]
    del train_albert_embedded
    gc.collect()
    return train

# TRAIN MODEL

In [15]:
import warnings
warnings.filterwarnings("ignore")
# betas = (0.5, 0.999)
lr = 0.0001
weight_decay = 0

model_ft = Model(config).to(torch.device("cuda"))
model_ft.load_state_dict(torch.load('model/0_200000_attention_bilstm_parameter.pkl'))
params_to_update = list(model_ft.parameters())
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr)
model_ft.train()
print("starting")

starting


In [ ]:
def generate_train(train, allsizes):
    length = train.shape[0]
    for i in range(0, length, allsizes):
        yield train.iloc[i:i+allsizes, :]
        
def generate_batch_train(query_vec, 
                         label_text_vec, 
                         query_bert_vec, 
                         img_vec, pos_vec, 
                         padding_mask, 
                         lstm_seq_length, 
                         label,
                         allsizes, 
                         batch):
    length = allsizes*2
    for i in range(0, length, batch):
        yield query_vec[i:i+batch], label_text_vec[i:i+batch], query_bert_vec[i:i+batch], img_vec[i:i+batch], pos_vec[i:i+batch], padding_mask[i:i+batch], lstm_seq_length[i:i+batch], label[i:i+batch]
        
def cal_prec(predict,labels):
    prec=0
    for p, l in zip(predict,labels):
        if p>0.5 and l==1:
            prec+=1
        if p<=0.5 and l==0:
            prec+=1
            
    return prec/len(predict)

allsizes=1000
batch=250
epoch=50

for e in range(1, epoch):
    print('epoch:', e)
    for s in range(1, 3):
        train_albert_path='../albert/avgpool_albert_embedded_'+str(s)+'00000'
        train_data_path='data/temp_data_'+str(s)+'00000.pkl'
        train = load_train_data(train_albert_path, train_data_path)
        
        train=shuffle(train, random_state=2020)
        train.reset_index(drop=True, inplace=True)
        gc.collect()
        
        print('load '+str(s*100000)+' train samples----------------------')
    
        trained_samples=allsizes+(s-1)*100000
        for big_batch_train in generate_train(train, allsizes):
            print('----trained samples:', trained_samples)
            
            trained_samples += allsizes

            big_batch_train['label']=1

            # padding
            def padding_seq(sequence, length, max_length):
                if length >= max_length:
                    return sequence
                else:
                    return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
            big_batch_train['label_embedding']=big_batch_train.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
            big_batch_train['pos']=big_batch_train.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
            big_batch_train['query_embedding']=big_batch_train.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
            big_batch_train['features']=big_batch_train.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
            gc.collect()

            # generate negative samples
            negative_train = generate_negative_sample(big_batch_train)
            if len(negative_train)<len(big_batch_train):
                    negative_train=pd.concat((negative_train,
                                              negative_train.iloc[0:(len(big_batch_train)-len(negative_train)),:]),axis=0)

            big_batch_train = pd.concat((big_batch_train, negative_train), axis=0)
            big_batch_train.reset_index(drop=True, inplace=True)
            big_batch_train=shuffle(big_batch_train, random_state=2020)
            big_batch_train.reset_index(drop=True, inplace=True)

            del negative_train
            gc.collect()

            query_vec = torch.Tensor(big_batch_train['query_embedding'])
            lstm_seq_length = list(big_batch_train['num_query_char'])

            label_text_vec = torch.Tensor(big_batch_train['label_embedding'])
            query_bert_vec = torch.Tensor(big_batch_train['query_albert_embedding']).view(allsizes*2, 1, 768)

            img_vec = torch.Tensor(big_batch_train['features'])
            pos_vec = torch.Tensor(big_batch_train['pos'])

            label =torch.Tensor(big_batch_train['label'])

            padding_mask =[]
            for i in list(big_batch_train['num_box']):
                mask=[0]*(i+2) + [1]*(MAX_BOX-i)
                padding_mask.append(mask)
            padding_mask = torch.ByteTensor(padding_mask)
            gc.collect()

            # train model in per batch samples
            train_count = 0
            avg_loss = 0
            avg_prec = 0
            for i in generate_batch_train(query_vec,label_text_vec,query_bert_vec,img_vec,pos_vec,padding_mask,lstm_seq_length,label,allsizes, batch):
                b_query_vec, b_label_text_vec, b_query_bert_vec, b_img_vec, b_pos_vec, b_padding_mask, b_lstm_seq_length, b_label=i
                optimizer.zero_grad()
                prediction=model_ft(b_query_vec.cuda(), b_label_text_vec.cuda(), b_query_bert_vec.cuda(), b_img_vec.cuda(), 
                                    b_pos_vec.cuda(), b_padding_mask.cuda(), b_lstm_seq_length)

                loss = calc_loss(prediction, b_label.cuda())
                prec = cal_prec(prediction, b_label.cuda())

                avg_loss += loss.item()
                avg_prec += prec
                loss.backward()
                optimizer.step()
                train_count += 1

            print('epoch:', e, 'trained samples:', trained_samples, '----average loss:', avg_loss / train_count)
            print('epoch:', e, 'trained samples:', trained_samples, '----average prec:', avg_prec / train_count)


        del train
        gc.collect()
        
        torch.save(model_ft.state_dict(), 'model/' + str(e) + '_'+ str(s*100000) + '_attention_bilstm_parameter.pkl')
        print('model/' + str(e) + '_' + str(s*100000) + '_attention_bilstm_parameter.pkl has been saved!')
        

epoch: 1



load 100000 train samples----------------------
----trained samples: 1000



negative sample number: 1000
epoch: 1 trained samples: 2000 ----average loss: 0.19538112916052341
epoch: 1 trained samples: 2000 ----average prec: 0.9235
----trained samples: 2000



negative sample number: 1000
epoch: 1 trained samples: 3000 ----average loss: 0.20531382225453854
epoch: 1 trained samples: 3000 ----average prec: 0.9190000000000002
----trained samples: 3000



negative sample number: 1000
epoch: 1 trained samples: 4000 ----average loss: 0.20520890317857265
epoch: 1 trained samples: 4000 ----average prec: 0.9230000000000002
----trained samples: 4000


...


negative sample number: 999
epoch: 1 trained samples: 64000 ----average loss: 0.1766745988279581
epoch: 1 trained samples: 64000 ----average prec: 0.938
----trained samples: 64000



negative sample number: 1000
epoch: 1 trained samples: 65000 ----average loss: 0.16218827106058598
epoch: 1 trained samples: 65000 ----average prec: 0.9390000000000001
----trained samples: 65000



negative sample number: 999
epoch: 1 trained samples: 66000 ----average loss: 0.16258556954562664
epoch: 1 trained samples: 66000 ----average prec: 0.946
----trained samples: 66000



negative sample number: 1000
epoch: 1 trained samples: 67000 ----average loss: 0.15632114093750715
epoch: 1 trained samples: 67000 ----average prec: 0.942
----trained samples: 67000



negative sample number: 1000
epoch: 1 trained samples: 68000 ----average loss: 0.1663501476868987
epoch: 1 trained samples: 68000 ----average prec: 0.939
----trained samples: 68000


In [55]:
print(query_vec[0:10].shape, 
      label_text_vec[0:10].shape, 
      query_bert_vec[0:10].shape, 
      img_vec[0:10].shape, 
      pos_vec[0:10].shape, 
      np.array(padding_mask[0:10]).shape, 
      np.array(lstm_seq_length[0:10]).shape)

query_vec1=torch.Tensor(query_vec[0:10].numpy().astype('float16'))
label_text_vec1=torch.Tensor(label_text_vec[0:10].numpy().astype('float16'))
query_bert_vec1=torch.Tensor(query_bert_vec[0:10].numpy().astype('float16'))
img_vec1=torch.Tensor(img_vec[0:10].numpy().astype('float16'))
pos_vec1=torch.Tensor(pos_vec[0:10].numpy().astype('float16'))


labels=torch.Tensor(big_batch_train['label'])[:10]
for i in range(100):
    
    optimizer.zero_grad()
    prediction=model_ft(query_vec1.cuda(), label_text_vec1.cuda(), query_bert_vec1.cuda(), 
                 img_vec1.cuda(), pos_vec1.cuda(), torch.ByteTensor(padding_mask[0:10]).cuda(), lstm_seq_length[0:10])

    loss = calc_loss(prediction, labels.cuda())
    print(str(i)+':loss-------',loss)

#             print('train prec:', cal_prec(predict,labels))

    loss.backward()
    optimizer.step()

torch.Size([10, 12, 600]) torch.Size([10, 10, 600]) torch.Size([10, 1, 768]) torch.Size([10, 10, 2048]) torch.Size([10, 10, 5]) (10, 12) (10,)
0:loss------- tensor(0.6204, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
1:loss------- tensor(0.4574, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
2:loss------- tensor(0.3725, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
3:loss------- tensor(0.3211, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
4:loss------- tensor(0.2872, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
5:loss------- tensor(0.2631, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
6:loss------- tensor(0.2458, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
7:loss------- tensor(0.2322, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
8:loss------- tensor(0.2214, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
9:loss-----

# VALID DATA

In [10]:
# load data
valid_albert_embedding=joblib.load('data/valid_query_albert.pkl')
valid=joblib.load('data/valid_data.pkl')
# truncate
valid['label_embedding']=valid.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
valid['pos']=valid.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
valid['query_embedding']=valid.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
valid['num_box']=valid.swifter.apply(lambda x: len(x['pos']),axis=1)
valid['num_query_char']=valid.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
valid['features']=valid.swifter.apply(lambda x: [i for i in x['features']],axis=1)
valid['features']=valid.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
# padding
def padding_seq(sequence, length, max_length):
    if length >= max_length:
        return sequence
    else:
        return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
    
valid['label_embedding']=valid.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
valid['pos']=valid.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
valid['query_embedding']=valid.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
valid['features']=valid.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
valid['query_albert_embedding']=valid_albert_embedding['text_vec']
gc.collect()
valid

,product_id,query_id,features,label_embedding,query_embedding,pos,num_box,num_query_char,query_albert_embedding
0,103047314,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.105, 0.4175, 0.1425, 0.59625, 0.1417966534...",2,3,"[0.73320353, -0.7702514, 0.52037656, -0.972297..."
1,103030678,0,"[[0.0, 0.0, 0.0022290119, 0.0, 0.0, 0.18552563...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.52375, 0.97375, 0.00625, 0.29125, 0.128249...",9,3,"[0.73320353, -0.7702514, 0.52037656, -0.972297..."
2,103031217,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.264, 0.564, 0.09221311475409837, 0.9672131...",2,3,"[0.73320353, -0.7702514, 0.52037656, -0.972297..."
3,103026621,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.4325, 0.63125, 0.566200215285253, 0.725511...",10,3,"[0.73320353, -0.7702514, 0.52037656, -0.972297..."
4,103017048,0,"[[0.0, 0.0, 0.05747965, 0.0, 0.0, 0.0, 0.0, 0....","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.2375, 0.6975, 0.34875, 0.90375, 0.25529960...",2,3,"[0.73320353, -0.7702514, 0.52037656, -0.972297..."
...,...,...,...,...,...,...,...,...,...
14715,103044330,500,"[[0.0, 0.0, 0.027551055, 0.0, 0.32401124, 0.0,...","[[-0.19830322, 0.09790802, 0.42492676, -0.3308...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.24375, 0.41625, 0.04875, 0.23875, 0.032774...",5,3,"[0.59812593, -0.666215, 0.56656677, -0.9123924..."
14716,103016765,500,"[[0.0, 0.0, 4.595919, 0.0, 0.0, 0.03517687, 0....","[[0.107177734, -0.19818115, -0.11024475, 0.392...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.16533333333333333, 1.3266666666666667, 0.1...",3,3,"[0.59812593, -0.666215, 0.56656677, -0.9123924..."
14717,103013406,500,"[[0.017800761, 0.0, 0.050141126, 0.0, 0.0, 0.0...","[[0.10523733, -0.16297404, -0.12767537, -0.051...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.28, 1.4893333333333334, 0.0, 0.66666666666...",3,3,"[0.59812593, -0.666215, 0.56656677, -0.9123924..."
14718,103012098,500,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.135, 0.211, 0.888, 0.996, 0.00820799179200...",6,3,"[0.59812593, -0.666215, 0.56656677, -0.9123924..."


In [12]:
# predict

import warnings
warnings.filterwarnings("ignore")

model_ft = Model(config).to(torch.device("cpu"))
model_ft.load_state_dict(torch.load('model/0_200000_attention_bilstm_parameter.pkl'))
model_ft.eval()

query_vec = torch.Tensor(valid['query_embedding'])
lstm_seq_length = list(valid['num_query_char'])
label_text_vec = torch.Tensor(valid['label_embedding'])
query_bert_vec = torch.Tensor(valid['query_albert_embedding']).view(14720, 1, 768)
img_vec = torch.Tensor(valid['features'])
pos_vec = torch.Tensor(valid['pos'])

padding_mask =[]
for i in list(valid['num_box']):
    mask=[0]*(i+2) + [1]*(MAX_BOX-i)
    padding_mask.append(mask)
padding_mask = torch.ByteTensor(padding_mask)
gc.collect()
        
# prediction=model_ft(query_vec.cuda(), label_text_vec.cuda(), query_bert_vec.cuda(), img_vec.cuda(), 
#                     pos_vec.cuda(), padding_mask.cuda(), lstm_seq_length)
print('start')
res = []
for i in range(0, 14720, 1000):
    print(i)
    prediction=model_ft(query_vec[i:i+1000], label_text_vec[i:i+1000], query_bert_vec[i:i+1000], img_vec[i:i+1000], 
                        pos_vec[i:i+1000], padding_mask[i:i+1000], lstm_seq_length[i:i+1000])
    res += [i[0].item() for i in prediction.detach().numpy()]
    gc.collect()

valid['prediction']=res
sorted_valid=valid.sort_values(by=['query_id','prediction'],ascending=[True, False])
submission=[]
for i in valid['query_id'].unique():
    submission.append([i]+list(sorted_valid[sorted_valid['query_id']==i]['product_id'].iloc[:5]))
print(submission)
submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
                                             'product3','product4','product5'],dtype=np.int)

submission
submission.to_csv('valid_pred.csv', index=0)

start
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
[[0, 103059361, 103017048, 103028493, 103020398, 103031217], [1, 103009387, 103065134, 103022099, 103049171, 103032961], [2, 103059018, 103059469, 103057259, 103039546, 103039137], [3, 103062592, 103040728, 103034297, 103046951, 103064331], [4, 103041391, 103042861, 103048149, 103028489, 103048840], [5, 103054044, 103033019, 103019152, 103007847, 103007999], [6, 103017342, 103038649, 103029444, 103000072, 103041148], [7, 103013757, 103041020, 103029770, 103023432, 103047484], [8, 103065668, 103040021, 103007787, 103061603, 103049972], [9, 103033259, 103040021, 103063825, 103061603, 103010304], [10, 103039501, 103028931, 103045719, 103049707, 103018933], [11, 103008253, 103049300, 103036002, 103008494, 103042885], [12, 103008191, 103018646, 103004892, 103042584, 103005351], [13, 103056700, 103004351, 103029379, 103056822, 103039899], [14, 103050414, 103038336, 103029023, 103012854, 103014269], [15, 103049

# TEST DATA

In [7]:
test_albert_embedding=joblib.load('data/test_query_albert.pkl')
test=joblib.load('data/test_data.pkl')
# truncate
test['label_embedding']=test.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
test['pos']=test.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
test['query_embedding']=test.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
test['num_box']=test.swifter.apply(lambda x: len(x['pos']),axis=1)
test['num_query_char']=test.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
test['features']=test.swifter.apply(lambda x: [i for i in x['features']],axis=1)
test['features']=test.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
# padding
def padding_seq(sequence, length, max_length):
    if length >= max_length:
        return sequence
    else:
        return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
    
test['label_embedding']=test.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
test['pos']=test.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
test['query_embedding']=test.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
test['features']=test.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
test['query_albert_embedding']=test_albert_embedding['text_vec']
gc.collect()

12

In [8]:
import warnings
warnings.filterwarnings("ignore")

model_ft = Model(config).to(torch.device("cpu"))
model_ft.load_state_dict(torch.load('model/9_1000000_attention_bilstm_parameter.pkl'))
model_ft.eval()

query_vec = torch.Tensor(test['query_embedding'])
lstm_seq_length = list(test['num_query_char'])
label_text_vec = torch.Tensor(test['label_embedding'])
query_bert_vec = torch.Tensor(test['query_albert_embedding']).view(28830, 1, 768)
img_vec = torch.Tensor(test['features'])
pos_vec = torch.Tensor(test['pos'])

padding_mask =[]
for i in list(test['num_box']):
    mask=[0]*(i+2) + [1]*(MAX_BOX-i)
    padding_mask.append(mask)
padding_mask = torch.ByteTensor(padding_mask)
gc.collect()

print('start')
res = []
for i in range(0, 28830, 1000):
    print(i)
    prediction=model_ft(query_vec[i:i+1000], label_text_vec[i:i+1000], query_bert_vec[i:i+1000], img_vec[i:i+1000], 
                        pos_vec[i:i+1000], padding_mask[i:i+1000], lstm_seq_length[i:i+1000])
    res += [i[0].item() for i in prediction.detach().numpy()]
    gc.collect()

test['prediction']=res
        
# prediction=model_ft(query_vec, label_text_vec, query_bert_vec, img_vec, 
#                     pos_vec, padding_mask, lstm_seq_length)

# test['prediction']=[i[0].item() for i in prediction.detach().numpy()]
sorted_test=test.sort_values(by=['query_id','prediction'],ascending=[True, False])
submission=[]
for i in test['query_id'].unique():
    submission.append([i]+list(sorted_test[sorted_test['query_id']==i]['product_id'].iloc[:5]))
submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
                                             'product3','product4','product5'],dtype=np.int)

submission
submission.to_csv('submission/single_tower_attention_bilstm_8_1000000_05220926_sub.csv', index=0)


start
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [14]:
del sorted_valid
gc.collect()

0